In [ ]:
import os
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import json
import time
import threading
import requests
from requests.auth import HTTPBasicAuth
from openai import OpenAI

data_file = "equities.json"

def strip_invisible(s: str) -> str:
    if not isinstance(s, str):
        return s
    return s.replace("\u200b", "").replace("\ufeff", "")

openai_key = "OPENAI_API_KEY" # Replace with actual key
client = OpenAI(api_key=openai_key) if openai_key else None

T212_KEY = "TRADING_212_API_KEY" # Replace with actual key
T212_SECRET = "TRADING_212_API_SECRET" #Replace with actual secret key
T212_BASE_URL = "https://demo.trading212.com/api/v0"

t212_auth = HTTPBasicAuth(T212_KEY, T212_SECRET)
t212_session = requests.Session()

def t212_request(method: str, path: str, *, params=None, payload=None):
    url = f"{T212_BASE_URL}{path}"
    r = t212_session.request(
        method=method,
        url=url,
        params=params,
        json=payload,
        auth=t212_auth,
        timeout=15,
    )
    if not r.ok:
        raise RuntimeError(f"T212 API error {r.status_code}: {r.text}")
    if r.status_code == 204:
        return None
    return r.json()


def t212_search_instruments(query: str, limit: int = 10):
    return t212_request("GET", "/equity/metadata/instruments", params={"search": query, "limit": limit}) or []

def t212_resolve_ticker(user_input: str) -> str:
    q = user_input.strip().upper()
    if not q:
        raise ValueError("Empty ticker")
    try:
        inst = t212_request("GET", f"/equity/metadata/instruments/{q}")
        # If it returned an object, accept it
        if isinstance(inst, dict):
            t = inst.get("ticker") or q
            return str(t)
    except Exception:
        pass

    results = t212_search_instruments(q, limit=10)
    if not results:
        raise ValueError(f"Ticker not found on Trading 212: {q}")
    first = results[0]
    ticker = first.get("ticker") or first.get("symbol") or first.get("id")
    if not ticker:
        raise ValueError(f"Unexpected instrument payload from Trading 212 for query {q}: {first}")

    return str(ticker)

def fetch_open_orders():
    try:
        orders = t212_request("GET", "/equity/orders") or []
        open_orders = []
        for o in orders:
            qty = o.get("quantity")
            qty_f = float(qty) if qty is not None else 0.0
            open_orders.append({
                "symbol": o.get("ticker"),
                "qty": qty,
                "limit_price": o.get("limitPrice"),
                "side": "sell" if qty_f < 0 else "buy",
            })
        return open_orders
    except Exception as e:
        print(strip_invisible(f"Error fetching open orders: {e}"))
        return []

def fetch_portfolio():
    try:
        positions = t212_request("GET", "/equity/positions") or []
        portfolio = []
        for p in positions:
            portfolio.append({
                "symbol": p.get("ticker"),
                "qty": p.get("quantity"),
                "entry_price": p.get("averagePricePaid"),
                "unrealized_pl": p.get("ppl"),
                "current_price": p.get("currentPrice"),
                "side": "long",
            })
        return portfolio
    except Exception as e:
        print(strip_invisible(f"Error fetching portfolio: {e}"))
        return []

def chatgpt_response(message: str) -> str:
    if client is None:
        return "OpenAI is not configured. Set OPENAI_API_KEY to enable AI responses."

    try:
        portfolio_data = fetch_portfolio()
        open_orders = fetch_open_orders()

        system_prompt = (
            "You are an AI Portfolio Manager. "
            "Your tasks are: "
            "1) Evaluate risk exposures of current holdings; "
            "2) Analyse open limit orders and their potential impact; "
            "3) Provide insights on portfolio health, diversification, and trade adjustments; "
            "4) Comment on market outlook based on general conditions (no real-time claims); "
            "5) Identify risks and suggest risk management strategies."
        )

        user_prompt = (
            f"Portfolio data (JSON-like): {portfolio_data}\n"
            f"Open orders (JSON-like): {open_orders}\n\n"
            f"User question: {message}"
        )

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.3,
        )
        return response.choices[0].message.content

    except Exception as e:
        return f"Error getting AI response: {e}"

class TradingBotGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Trading Bot")
        self.equities = self.load_equities()
        self.system_running = False

        self.form_frame = tk.Frame(root)
        self.form_frame.pack(pady=10)

        tk.Label(self.form_frame, text="Symbol:").grid(row=0, column=0, padx=5)
        self.symbol_entry = tk.Entry(self.form_frame)
        self.symbol_entry.grid(row=0, column=1, padx=5)

        tk.Label(self.form_frame, text="Levels:").grid(row=0, column=2, padx=5)
        self.levels_entry = tk.Entry(self.form_frame)
        self.levels_entry.grid(row=0, column=3, padx=5)

        tk.Label(self.form_frame, text="Drawdown%:").grid(row=0, column=4, padx=5)
        self.drawdown_entry = tk.Entry(self.form_frame)
        self.drawdown_entry.grid(row=0, column=5, padx=5)

        self.add_button = tk.Button(self.form_frame, text="Add Equity", command=self.add_equity)
        self.add_button.grid(row=0, column=6, padx=5)

        self.tree = ttk.Treeview(root, columns=("Symbol", "Position", "Entry Price", "Levels", "Status"), show="headings")
        for col in ["Symbol", "Position", "Entry Price", "Levels", "Status"]:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=120)
        self.tree.pack(pady=10)

        self.toggle_selected_system_button = tk.Button(root, text="Toggle Selected System", command=self.toggle_selected_system)
        self.toggle_selected_system_button.pack(pady=5)

        self.remove_button = tk.Button(root, text="Remove Selected Equity", command=self.remove_equity)
        self.remove_button.pack(pady=5)

        # AI component
        self.chat_frame = tk.Frame(root)
        self.chat_frame.pack(pady=10)

        self.chat_input = tk.Entry(self.chat_frame, width=50)
        self.chat_input.grid(row=0, column=0, padx=5)

        self.send_button = tk.Button(self.chat_frame, text="Send", command=self.send_message)
        self.send_button.grid(row=0, column=1, padx=5)

        self.chat_output = tk.Text(root, height=10, width=60, state=tk.DISABLED)
        self.chat_output.pack(pady=10)

        self.refresh_table()

        self.running = True
        self.auto_update_thread = threading.Thread(target=self.auto_update, daemon=True)
        self.auto_update_thread.start()

    def add_equity(self):
        user_input = self.symbol_entry.get().strip()
        levels = self.levels_entry.get().strip()
        drawdown = self.drawdown_entry.get().strip()

        if not user_input or not levels.isdigit() or not drawdown.replace(".", "", 1).isdigit():
            messagebox.showerror("Error", "Invalid Input")
            return
        try:
            symbol = t212_resolve_ticker(user_input)
        except Exception as e:
            messagebox.showerror("Ticker Error", str(e))
            return

        levels = int(levels)
        drawdown = float(drawdown) / 100.0
        level_placeholders = {i + 1: None for i in range(levels)}

        self.equities[symbol] = {
            "position": 0,
            "entry_price": None,
            "levels": level_placeholders,
            "drawdown": drawdown,
            "status": "Off",
        }

        self.save_equities()
        self.refresh_table()

    def toggle_selected_system(self):
        selected_items = self.tree.selection()
        if not selected_items:
            messagebox.showerror("Error", "No Equity Selected")
            return

        for item in selected_items:
            symbol = self.tree.item(item)["values"][0]
            self.equities[symbol]["status"] = "On" if self.equities[symbol]["status"] == "Off" else "Off"

        self.save_equities()
        self.refresh_table()

    def remove_equity(self):
        selected_items = self.tree.selection()
        if not selected_items:
            messagebox.showerror("Error", "No Equity Selected")
            return

        for item in selected_items:
            symbol = self.tree.item(item)["values"][0]
            if symbol in self.equities:
                del self.equities[symbol]

        self.save_equities()
        self.refresh_table()

    def send_message(self):
        message = self.chat_input.get()
        if not message:
            return
            
        response = chatgpt_response(message)

        self.chat_output.config(state=tk.NORMAL)
        self.chat_output.insert(tk.END, f"You: {message}\n{response}\n\n")
        self.chat_output.config(state=tk.DISABLED)
        self.chat_input.delete(0, tk.END)

    def _append_chat(self, message, response):
        self.chat_output.config(state=tk.NORMAL)
        self.chat_output.insert(tk.END, f"You: {message}\n{response}\n\n")
        self.chat_output.config(state=tk.DISABLED)

    def fetch_market_data(self, symbol):
        return {"price": -1}

    def trade_systems(self):
        for symbol, data in self.equities.items():
            if data.get("status") != "On":
                continue
            try:
                positions = t212_request("GET", "/equity/positions", params={"ticker": symbol}) or []
            except Exception as e:
                print(strip_invisible(f"T212 position fetch failed for {symbol}: {e}"))
                continue

            if not positions:
                try:
                    t212_request("POST", "/equity/orders/market", payload={"ticker": symbol, "quantity": 1})
                except Exception as e:
                    print(strip_invisible(f"T212 market buy failed for {symbol}: {e}"))
                    continue

                time.sleep(1)
                try:
                    positions = t212_request("GET", "/equity/positions", params={"ticker": symbol}) or []
                except Exception as e:
                    print(strip_invisible(f"T212 position re-fetch failed for {symbol}: {e}"))
                    continue

                if not positions:
                    print(strip_invisible(f"No position returned after market buy for {symbol}."))
                    continue

            position = positions[0]
            qty = float(position.get("quantity") or 0.0)
            entry_price = float(position.get("averagePricePaid") or 0.0)

            self.equities[symbol]["position"] = qty
            self.equities[symbol]["entry_price"] = entry_price

            existing_levels = self.equities[symbol].get("levels", {})
            if not isinstance(existing_levels, dict):
                existing_levels = {}

            normalised_levels = {}
            for k, v in existing_levels.items():
                try:
                    normalised_levels[int(k)] = v
                except Exception:
                    pass
            existing_levels = normalised_levels

            n_levels = len(existing_levels)
            if n_levels == 0:
                n_levels = 1
                existing_levels = {1: None}

            drawdown = float(self.equities[symbol].get("drawdown", 0.0))

            level_prices = {
                i + 1: round(entry_price * (1 - drawdown * (i + 1)), 2)
                for i in range(n_levels)
            }

            for level, price in level_prices.items():
                if level not in existing_levels or existing_levels[level] is None:
                    existing_levels[level] = price

            self.equities[symbol]["levels"] = existing_levels

            for level, price in list(existing_levels.items()):
                if price is None:
                    continue
                try:
                    self.place_order(symbol, float(price), int(level))
                except Exception as e:
                    print(strip_invisible(f"Failed placing order {symbol} L{level} @ {price}: {e}"))

            self.save_equities()
            self.refresh_table()

    def place_order(self, symbol, price, level):
        levels_dict = self.equities[symbol].get("levels", {})
        if not isinstance(levels_dict, dict):
            return

        levels_dict = {int(k): v for k, v in levels_dict.items() if str(k).isdigit()}

        if level not in levels_dict:
            return

        t212_request("POST", "/equity/orders/limit", payload={
            "ticker": symbol,
            "quantity": -1,
            "limitPrice": float(price),
            "timeValidity": "GTC",
        })

        del levels_dict[level]
        self.equities[symbol]["levels"] = levels_dict

        print(strip_invisible(f"Placed limit sell for 1 of {symbol} at {price} (level {level})."))

    def refresh_table(self):
        for row in self.tree.get_children():
            self.tree.delete(row)

        for symbol, data in self.equities.items():
            levels_dict = data.get("levels", {})
            levels_display = len(levels_dict) if isinstance(levels_dict, dict) else 0

            self.tree.insert("", tk.END, values=(
                symbol,
                data.get("position", 0),
                data.get("entry_price", ""),
                levels_display,
                data.get("status", "Off"),
            ))

    def auto_update(self):
        while self.running:
            time.sleep(5)
            self.trade_systems()

    def save_equities(self):
        with open(data_file, "w") as f:
            json.dump(self.equities, f)

    def load_equities(self):
        try:
            with open(data_file, "r") as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            return {}

    def on_close(self):
        self.running = False
        self.save_equities()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = TradingBotGUI(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()